# The construction of transmit waveforms
> **Warning:** This interactive notebook does not work properly in Google Colab. To get proper functionality you can download it and run it locally.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import h5py
import requests
import ipywidgets

# ======================================================================================
# Download the darkmode.mplstyle stylesheet and use it
# ======================================================================================
# Download the darkmode.mplstyle stylesheet from the website repository
url = (
    r"https://raw.githubusercontent.com/vincentvdschaft/quartz-website/v4/"
    r"figure-generation/darkmode.mplstyle"
)
r = requests.get(url)
# Write the downloaded stylesheet to a file
with open("stylesheet.mplstyle", "wb") as f:
    f.write(r.content)
# Use the stylesheet
plt.style.use("stylesheet.mplstyle")

# Get the default color cycle
color_cycle = plt.rcParams["axes.prop_cycle"].by_key()["color"]

## Focused wavefront

In [ ]:
# Define the x-coordinates of the elements in meters
element_x = np.linspace(-20e-3, 20e-3, 16)

# Define the speed of sound in m/s
c = 1540

zlim = (-20e-3, 50e-3)
xlim = (-22e-3, 22e-3)


def plot_focused_wavefront(fig, ax, tus, x0, z0, draw_lines):
    """Plots a visualization of how a focused wavefront is constructed from a linear
    array of elements.

    Parameters
    ----------
    fig : mpl.figure.Figure
        The figure to plot to.
    ax : mpl.axes.Axes
        The axes to plot to.
    tus : float
        The time instant at which the wavefront is visualized in microseconds.
    x0 : float
        The x-coordinate of the virtual source in meters.
    z0 : float
        The z-coordinate of the virtual source in meters.
    draw_lines : bool
        Set to True to draw the lines indicating the wave cone.
    """
    t = (tus - 30) * 1e-6
    x0 = x0 * 1e-3
    z0 = z0 * 1e-3

    # Draw the lines indicating the wave cone
    if draw_lines:
        dxdz = (x0 - element_x[0]) / z0
        ax.plot([element_x[0], element_x[0] + dxdz], [0, 1], "--", color="gray")
        dxdz = (x0 - element_x[-1]) / z0
        ax.plot([element_x[-1], element_x[-1] + dxdz], [0, 1], "--", color="gray")

        # Define vector in the direction of the virtual source
        v = np.array([x0, z0])
        v = v / np.linalg.norm(v)

        if t > 0:
            v = -v

        # Draw tangent line of the virtual wavefront
        ax.plot(
            [x0 - v[0] * np.abs(c * t) + v[1], x0 - v[0] * np.abs(c * t) - v[1]],
            [z0 - v[1] * np.abs(c * t) - v[0], z0 - v[1] * np.abs(c * t) + v[0]],
            "w--",
        )

    # Draw the elements
    ax.plot(element_x, 0 * element_x, "o", markersize=3, color="white")

    ax.set_xlabel("x [mm]")
    ax.set_ylabel("z [mm]")

    # Draw the virtual source
    ax.plot(x0, z0, "o", color=color_cycle[2])

    # Draw the wavefronts from all elements
    t0_arr = np.zeros_like(element_x)
    for n, x in enumerate(element_x):
        t0 = -np.sqrt((x - x0) ** 2 + z0**2) / c
        t0_arr[n] = t0
        if t > t0:
            ax.add_patch(
                matplotlib.patches.Circle(
                    (x, 0),
                    radius=np.abs(c * (t - t0)),
                    fill=False,
                    color=color_cycle[0],
                )
            )

    # Draw the virual wavefront
    ax.add_patch(
        matplotlib.patches.Circle(
            (x0, z0), radius=np.abs(c * t), fill=False, color="gray", linestyle="--"
        ),
    )

    # Define axis tick formatter to convert to mm
    def mm(x, pos):
        return f"{x*1000:.0f}"

    ax.xaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(mm))
    ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(mm))

    # Configure the plot
    ax.set_ylim(zlim)
    ax.set_xlim((xlim[0], xlim[1]))
    ax.set_aspect("equal")
    ax.set_title("focused wavefront")


# ======================================================================================
# Create an interactive plot
# ======================================================================================
def make_plot(tus, x0, z0, draw_lines):

    fig, ax = plt.subplots(figsize=(7, 4))

    plot_focused_wavefront(fig, ax, tus, x0, z0, draw_lines)

    return fig, ax


slider_tus = ipywidgets.FloatSlider(
    min=0, max=160, step=1, value=23, description="t [us]"
)
slider_x = ipywidgets.FloatSlider(
    min=-20, max=20, step=0.5, value=0, description="x [mm]"
)
slider_z = ipywidgets.FloatSlider(
    min=-0, max=65, step=1, value=20, description="z [mm]"
)

checkbox = ipywidgets.Checkbox(value=False, description="Draw lines")

interactive_plot = ipywidgets.interactive(
    make_plot, tus=slider_tus, x0=slider_x, z0=slider_z, draw_lines=checkbox
)


interactive_plot

## Diverging wavefront

In [ ]:
def plot_diverging_wavefront(fig, ax, tus, x0, z0):
    """Plots a visualization of how a diverging wavefront is constructed from a linear
    array of elements.

    Parameters
    ----------
    fig : mpl.figure.Figure
        The figure to plot to.
    ax : mpl.axes.Axes
        The axes to plot to.
    tus : float
        The time instant at which the wavefront is visualized in microseconds.
    x0 : float
        The x-coordinate of the virtual source in meters.
    z0 : float
        The z-coordinate of the virtual source in meters.
    """

    # Convert the input to base units
    t = tus * 1e-6
    x0 = x0 * 1e-3
    z0 = z0 * 1e-3

    # Draw the elements
    ax.plot(element_x, 0 * element_x, "o", markersize=3, color="white")
    ax.set_xlabel("x [mm]")
    ax.set_ylabel("z [mm]")

    # Draw the virtual source
    ax.plot(x0, z0, "o", color=color_cycle[2])

    # Draw the wavefronts from all elements
    for x in element_x:
        t0 = np.sqrt((x - x0) ** 2 + z0**2) / c
        if t > t0:
            circle = matplotlib.patches.Circle(
                    (x, 0),
                    radius=np.abs(c * (t - t0)),
                    fill=False,
                    color=color_cycle[0],
                    label="wavefront",
                )
            ax.add_patch(
                circle
            )

    # Draw the virual wavefront
    ax.add_patch(
        matplotlib.patches.Circle(
            (x0, z0), radius=np.abs(c * t), fill=False, color="gray", linestyle="--"
        ),
    )

    # Define axis tick formatter to convert to mm
    def mm(x, pos):
        return f"{x*1000:.0f}"
    ax.xaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(mm))
    ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(mm))

    # Configure the plot
    ax.set_ylim(zlim)
    ax.set_xlim((xlim[0], xlim[1]))
    ax.set_aspect("equal")
    ax.set_title("diverging wavefront")


# ======================================================================================
# Create the interactive plot
# ======================================================================================


def make_plot(tus, x0, z0):
    """Creates an interactive plot of a diverging wavefront.

    Parameters
    ----------
    tus : float
        The time instant at which the wavefront is visualized in microseconds.
    x0 : float
        The x-coordinate of the virtual source in mm.
    z0 : float
        The z-coordinate of the virtual source in mm.
    """

    fig, ax = plt.subplots(figsize=(7, 4))

    plot_diverging_wavefront(fig, ax, tus, x0, z0)

    return fig, ax


# ======================================================================================
# Define the input sliders
# ======================================================================================
slider_tus = ipywidgets.FloatSlider(
    min=0, max=60, step=1, value=20, description="t [us]"
)
slider_x = ipywidgets.FloatSlider(
    min=-20, max=20, step=0.5, value=0, description="x [mm]"
)
slider_z = ipywidgets.FloatSlider(
    min=-20, max=0, step=1, value=-15, description="z [mm]"
)

# ======================================================================================
# Set up the interactive plot
# ======================================================================================
interactive_plot = ipywidgets.interactive(
    make_plot, tus=slider_tus, x0=slider_x, z0=slider_z
)


interactive_plot

## Planar wavefront

In [ ]:
def plot_planar_wavefront(fig, ax, tus, theta):
    """Plots a visualization of the construction of a planar wavefront from a linear
    array of elements.

    Parameters
    ----------
    fig : mpl.figure.Figure
        The figure to plot to.
    ax : mpl.axes.Axes
        The axes to plot to.
    tus : float
        The time instant at which the wavefront is visualized in microseconds.
    theta : float
        The angle of the virtual source in degrees.
    """

    # Convert the input to base units
    t = (tus-18) * 1e-6
    theta = np.deg2rad(theta)

    # Draw the elements
    ax.plot(element_x, 0 * element_x, "o", markersize=3, color="white", label="elements")
    ax.set_xlabel("x [mm]")
    ax.set_ylabel("z [mm]")

    # Draw the virtual source direction
    # Define vector in the direction of the virtual source at infinity
    v = np.array([np.sin(theta), np.cos(theta)])
    r = 150e-3
    ax.plot([0, r * v[0]], [0, r * v[1]], "--", color=color_cycle[2])

    # Define a vector perpendicular to the virtual source direction
    wavefront_vec = np.array([v[1], -v[0]])
    # Define a vector pointing to the middle of the wavefront
    origin = v * t * c

    # Draw the virtual wavefront
    ax.plot(
        [origin[0] - r * wavefront_vec[0], origin[0] + r * wavefront_vec[0]],
        [origin[1] - r * wavefront_vec[1], origin[1] + r * wavefront_vec[1]],
        "--",
        color="gray"
    )

    # Draw the wavefronts from all elements
    for x in element_x:
        p = np.array([x, 0])
        t0 = np.dot(v, p) / c
        if t > t0:
            ax.add_patch(
                matplotlib.patches.Circle(
                    (x, 0),
                    radius=np.abs(c * (t - t0)),
                    fill=False,
                    color=color_cycle[0],
                )
            )

    # Define axis tick formatter to convert to mm
    def mm(x, pos):
        return f"{x*1000:.0f}"
    ax.xaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(mm))
    ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(mm))

    # Configure the plot
    ax.set_ylim(zlim)
    ax.set_xlim((xlim[0], xlim[1]))
    ax.set_aspect("equal")
    ax.set_title("planar wavefront")


# ======================================================================================
# Create the interactive plot
# ======================================================================================
def make_plot(tus, theta):
    """Creates an interactive plot of a planar wavefront.

    Parameters
    ----------
    tus : float
        The time instant at which the wavefront is visualized in microseconds.
    theta : float
        The angle of the virtual source in degrees.
    """
    fig, ax = plt.subplots(figsize=(7, 4))

    plot_planar_wavefront(fig, ax, tus, theta)

    return fig, ax


# ======================================================================================
# Define the input sliders
# ======================================================================================
slider_tus = ipywidgets.FloatSlider(
    min=0, max=110, step=1, value=14, description="t [us]"
)
slider_theta = ipywidgets.FloatSlider(
    min=-90, max=90, step=0.5, value=22, description="angle [deg]"
)

# ======================================================================================
# Set up the interactive plot
# ======================================================================================
interactive_plot = ipywidgets.interactive(make_plot, tus=slider_tus, theta=slider_theta)
interactive_plot

## Combine everything in a single figure

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(7, 4))
plot_focused_wavefront(fig, axes[0], 23, 0, 20, False)
plot_diverging_wavefront(fig, axes[1], 20, 0, -15)
plot_planar_wavefront(fig, axes[2], 23, 22)
plt.tight_layout()
plt.savefig("../content/assets/transmit_wavefronts.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
import glob
import os
from PIL import Image
temp_dir = Path("temp")
temp_dir.mkdir(exist_ok=True)

N_FRAMES = 90

for n in range(N_FRAMES):
    path = temp_dir / f"wavefronts_{str(n).zfill(3)}.png"

    fig, axes = plt.subplots(1, 3, figsize=(7, 4))
    plot_focused_wavefront(fig, axes[0], n, 0, 20, False)
    plot_diverging_wavefront(fig, axes[1], n, 0, -15)
    plot_planar_wavefront(fig, axes[2], n, 22)
    plt.tight_layout()
    plt.savefig(path, dpi=300, bbox_inches='tight')
    plt.close()


def pngs_to_gif(folder_path, output_filename, duration=500):
    """Converts all PNG images in the specified folder into a single GIF.

    Parameters
    ----------
    folder_path : str
        The path to the folder containing PNG images.
    output_filename : str
        The filename for the output GIF.
    duration : int
        Duration of each frame in the GIF in seconds.
    """
    images = []
    # Loop through all files in the folder
    image_paths = glob.glob(os.path.join(folder_path, "*.png"))
    # Sort the images by name
    image_paths.sort()
    for image_path in image_paths:
        images.append(Image.open(image_path))

    # Save the images as a GIF
    images[0].save(
        output_filename,
        save_all=True,
        append_images=images[1:],
        optimize=False,
        duration=duration,
        loop=0,
    )

pngs_to_gif("temp", "../content/assets/transmit_wavefronts.gif", duration=6)

# Clean up the temporary folder
for file in temp_dir.glob("*.png"):
    file.unlink()
# Remove the temporary folder
temp_dir.rmdir()